# Single-View Geometry (Python)

## Usage
This code snippet provides an overall code structure and some interactive plot interfaces for the *Single-View Geometry* section of Assignment 3. In [main function](#Main-function), we outline the required functionalities step by step. Some of the functions which involves interactive plots are already provided, but [the rest](#Your-implementation) are left for you to implement.

## Package installation
- You will need [GUI backend](https://matplotlib.org/faq/usage_faq.html#what-is-a-backend) to enable interactive plots in `matplotlib`.
- In this code, we use `tkinter` package. Installation instruction can be found [here](https://anaconda.org/anaconda/tk).

# Common imports

In [ ]:
% matplotlib tk
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import sympy
from sympy import Symbol, Matrix, solve
from PIL import Image
import pickle
import skimage
from skimage import transform


# Provided functions

In [ ]:
def get_input_lines(im, min_lines=3):
    """
    Allows user to input line segments; computes centers and directions.
    Inputs:
        im: np.ndarray of shape (height, width, 3)
        min_lines: minimum number of lines required
    Returns:
        n: number of lines from input
        lines: np.ndarray of shape (3, n)
            where each column denotes the parameters of the line equation
        centers: np.ndarray of shape (3, n)
            where each column denotes the homogeneous coordinates of the centers
    """
    n = 0
    lines = np.zeros((3, 0))
    centers = np.zeros((3, 0))

    plt.figure()
    plt.imshow(im)
    print('Set at least {} lines to compute vanishing point'.format(min_lines))
    while True:
        print('Click the two endpoints, use the right key to undo, and use the middle key to stop input')
        clicked = plt.ginput(2, timeout=0, show_clicks=True)
#         print(clicked)
        if not clicked or len(clicked) < 2:
            if n < min_lines:
                print('Need at least {} lines, you have {} now'.format(min_lines, n))
                continue
            else:
                # Stop getting lines if number of lines is enough
                break

        # Unpack user inputs and save as homogeneous coordinates
        pt1 = np.array([clicked[0][0], clicked[0][1], 1])
        pt2 = np.array([clicked[1][0], clicked[1][1], 1])
        # Get line equation using cross product
        # Line equation: line[0] * x + line[1] * y + line[2] = 0
        line = np.cross(pt1, pt2)
        lines = np.append(lines, line.reshape((3, 1)), axis=1)
        # Get center coordinate of the line segment
        center = (pt1 + pt2) / 2
        centers = np.append(centers, center.reshape((3, 1)), axis=1)

        # Plot line segment
        plt.plot([pt1[0], pt2[0]], [pt1[1], pt2[1]], color='b')

        n += 1

    return n, lines, centers

In [ ]:
def plot_lines_and_vp(ax, im, lines, vp):
    """
    Plots user-input lines and the calculated vanishing point.
    Inputs:
        im: np.ndarray of shape (height, width, 3)
        lines: np.ndarray of shape (3, n)
            where each column denotes the parameters of the line equation
        vp: np.ndarray of shape (3, )
    """
    bx1 = min(1, vp[0] / vp[2]) - 10
    bx2 = max(im.shape[1], vp[0] / vp[2]) + 10
    by1 = min(1, vp[1] / vp[2]) - 10
    by2 = max(im.shape[0], vp[1] / vp[2]) + 10
    
    ax.imshow(im)
    for i in range(lines.shape[1]):
        if lines[0, i] < lines[1, i]:
            pt1 = np.cross(np.array([1, 0, -bx1]), lines[:, i])
            pt2 = np.cross(np.array([1, 0, -bx2]), lines[:, i])
        else:
            pt1 = np.cross(np.array([0, 1, -by1]), lines[:, i])
            pt2 = np.cross(np.array([0, 1, -by2]), lines[:, i])
        pt1 = pt1 / pt1[2]
        pt2 = pt2 / pt2[2]
        ax.plot([pt1[0], pt2[0]], [pt1[1], pt2[1]], 'g')

    ax.plot(vp[0] / vp[2], vp[1] / vp[2], 'ro')
    ax.set_xlim([bx1, bx2])
    ax.set_ylim([by2, by1])

# Your implementation

In [ ]:
def get_vanishing_point(lines):
    """
    Solves for the vanishing point using the user-input lines.
    """
    l1 = lines[:,0] / np.sqrt(lines[:,0][0]**2 + lines[:,0][1]**2) 
    l2 = lines[:,1] / np.sqrt(lines[:,1][0]**2 + lines[:,1][1]**2)
    l3 = lines[:,2] / np.sqrt(lines[:,2][0]**2 + lines[:,2][1]**2)
    A = np.vstack((l1,l2,l3))
    A_t = A.T
    
#     a,b = np.linalg.eig(A_t.dot(A)) 
#     vp = b[:,np.argmin(a)]
    
    u, s, v = np.linalg.svd(A)
    vp = v[-1,:]
    vp /= vp[-1]
    
    
    
    return vp

In [ ]:
def get_horizon_line(vpts):
    """
    Calculates the ground horizon line.
    """
    p1 = vpts[:,0]
    p2 = vpts[:,1]
    
    horizon = np.cross(p1,p2)
    normal = np.sqrt(horizon[0]**2 + horizon[1]**2)
    horizon /= normal
    
    return horizon


In [ ]:
def plot_horizon_line(im, vpts, line):
    """
    Plots the horizon line.
    """
#     p1, p2 = vpts[:,0], vpts[:,1]
#     x1,y1,_ = p1
#     x2,y2,_ = p2
    
# #     plt.figure()
#     plt.imshow(im)
#     plt.plot([x1, x2], [y1, y2], color='y')
#     plt.plot(x1,y1,color='r',marker='D')
#     plt.plot(x2,y2,color='r',marker='D')

    h,w,_ = im.shape
    a,b,c = line
    y1 = - c/b
    y2 = - (c+w*a)/b
    
    ax.plot([0, w], [y1, y2], 'r')
    ax.imshow(im)

    pass
    

In [ ]:
def get_camera_parameters(vpts):
    """
    Computes the camera parameters. Hint: The SymPy package is suitable for this.
    """
    f = Symbol('f')
    u = Symbol('u')
    v = Symbol('v')
    p1 = Matrix(vpts[:,0]) 
    p2 = Matrix(vpts[:,1]) 
    p3 = Matrix(vpts[:,2])
    
    K = Matrix([[f, 0, u],[0, f, v],[0, 0, 1]])
    K_inv = K.inv()
    
    Eq1 = p1.T * K_inv.T * K_inv * p2
    Eq2 = p1.T * K_inv.T * K_inv * p3
    Eq3 = p2.T * K_inv.T * K_inv * p3
    
    ans = solve([Eq1, Eq2, Eq3], [f, u, v])
    
    f, u, v = ans[0]
    K = np.array([[f, 0, u], [0, f, v], [0, 0, 1]]).astype(np.float)
    return abs(f), u, v, K 



In [ ]:
def get_rotation_matrix(vpts, K):
    """
    Computes the rotation matrix using the camera parameters.
    """
    vx, vy, vz = vpts[:,1], vpts[:,2], vpts[:,0]
    K_inv = np.linalg.inv(K)
    
    r1 = K_inv.dot(vx)
    r2 = K_inv.dot(vy)
    r3 = K_inv.dot(vz)
    
    r1 /= np.linalg.norm(r1)
    r2 /= np.linalg.norm(r2)
    r3 /= np.linalg.norm(r3)
    
    R = np.vstack((r1, r2, r3)).T
 
    return R


In [ ]:
def get_homography(K,R):
    """
    Compute homography for transforming the image into fronto-parallel 
    views along the different axes.
    """
    K_inv = np.linalg.inv(K)
    H = K.dot(R).dot(K_inv)

    return H
    

In [ ]:
# def get_rotation_matrix_rectification(R, axis='z'):
#     """
#     Compute the rotation matrix that will be used to compute the 
#     homography for rectification.
#     """
#     if axis == 'z':
#         R_ = np.array([[1,0,0],[0,1,0],[0,0,1]])
#     elif axis == 'x':
#         R_ = np.array([[0,0,-1],[0,1,0],[1,0,0]])
#     elif axis == 'y':
#         R_ = np.array([[0,-1,0],[1,0,0],[0,0,-1]])
        
#     return R_.dot(np.linalg.inv(R))

In [ ]:
# def get_fronto_image(H,img):
#     transform = skimage.transform.ProjectiveTransform(H)
#     h, w, _ = img.shape
#     raw = np.array([[0, 0], [0, h], [w, h], [w, 0]])
#     trans = transform(raw)
    
#     min_dot = np.int32(trans.min(axis=0))
#     max_dot = np.int32(trans.max(axis=0))
#     offset_x = -min_dot[0]
#     offset_y = -min_dot[1]
#     print("offset is ", (offset_x, offset_y))
#     T_matrix = np.array([[1, 0, offset_x], [0, 1, offset_y], [0, 0, 1]])
    
#     fronto_image = cv2.warpPerspective(img, T_matrix.dot(H), tuple(max_dot - min_dot)) 
    
#     return fronto_image


# Main function

In [ ]:
im = np.asarray(Image.open('./data/Q3/eceb.png'))

# Also loads the vanishing line data if it exists in data.pickle file. 
# data.pickle is written using snippet in the next cell.
if os.path.exists('data.pickle'):
    with open('data.pickle', 'rb') as f:
        all_n, all_lines, all_centers = pickle.load(f, encoding='latin')
num_vpts = 3

In [ ]:
# # Click and save the line data for vanishing points. This snippet 
# # opens up an interface for selecting points and writes them to 
# # data.pickle file. The file is over-written.

# num_vpts = 3
# all_n, all_lines, all_centers = [], [], []

# for i in range(num_vpts):
#     print('Getting vanishing point {}'.format(i))
# #     fig = plt.figure()
# #     ax = fig.gca()
    
#     # Get at least three lines from user input
#     n_i, lines_i, centers_i = get_input_lines(im)
#     all_n.append(n_i)
#     all_lines.append(lines_i)
#     all_centers.append(centers_i)

# with open('data.pickle', 'wb') as f:
#     pickle.dump([all_n, all_lines, all_centers], f)

In [ ]:
# Part (a)
# Computing vanishing points for each of the directions
vpts = np.zeros((3, num_vpts))
for i in range(num_vpts):
    fig = plt.figure(); ax = fig.gca()
    
    vpts[:, i] = get_vanishing_point(all_lines[i])
    
    # Plot the lines and the vanishing point
    plot_lines_and_vp(ax, im, all_lines[i], vpts[:, i])
    fig.savefig('Q3_vp{:d}.pdf'.format(i), bbox_inches='tight')
print(vpts) 

In [ ]:
# Part (b) Computing and plotting the horizon

horizon_line = get_horizon_line(vpts)
print(horizon_line)
fig = plt.figure()
ax = fig.gca()
plot_horizon_line(im,vpts,horizon_line)
fig.savefig('Q3_horizon.pdf', bbox_inches='tight')

In [ ]:
# Part (c) Computing Camera Parameters
# <YOUR CODE> Solve for the camera parameters (f, u, v)
f, u, v, K = get_camera_parameters(vpts)
print("u, v, f, K are:")
print(u, v, f, K)

In [ ]:
# Part (d) Computing Rotation Matrices
R = get_rotation_matrix(vpts, K)
print(R)


In [ ]:
def get_rotation_matrix_rectification(R, axis='z'):
    """
    Compute the rotation matrix that will be used to compute the 
    homography for rectification.
    """
#     if axis == 'y':
#         R_ = np.array([[1,0,0],[0,0,1],[0,-1,0]])
#     elif axis == 'z':
#         R_ = np.array([[0,0,1],[0,1,0],[-1,0,0]])
#     elif axis == 'x':
#         R_ = np.array([[0,1,0],[1,0,0],[0,0,-1]])
        
    if axis == 'z':
        R_ = np.array([[1,0,0],[0,1,0],[0,0,1]])
    elif axis == 'x':
        R_ = np.array([[1,0,0],[0,0,-1],[0,-1,0]])
    elif axis == 'y':
        R_ = np.array([[0,0,1],[0,-1,0],[1,0,0]])

        
    return R_.dot(np.linalg.inv(R))

In [ ]:
def get_fronto_image(H,img):
    transform = skimage.transform.ProjectiveTransform(H)
    h, w, _ = img.shape
    raw = np.array([[0, 0], [0, h], [w, h], [w, 0]])
    trans = transform(raw)
    
    min_dot = np.int32(trans.min(axis=0))
    max_dot = np.int32(trans.max(axis=0))

    offset_x = -min_dot[0]
    offset_y = -min_dot[1]
    print("offset is ", (offset_x, offset_y))
    
    T_matrix = np.array([[1, 0, offset_x], [0, 1, offset_y], [0, 0, 1]])
    fronto_image = cv2.warpPerspective(img,  T_matrix.dot(H), 5000,4000) 
#     fronto_image = cv2.warpPerspective(img,  H, tuple(max_dot - min_dot))

    
    return fronto_image

# def get_fronto_image(H,img):
#     w =4000
#     h = 4000
    
#     fronto_image = cv2.warpPerspective(img, H, (w,h)) 
#     fronto_image = cv2.cvtColor(fronto_image, cv2.COLOR_BGR2RGB)
    
#     return fronto_image


In [ ]:
# Part (e) Generating fronto-parallel warps. Compute the 
# appropriate rotation to transform the world coordinates
# such that the axis of projection becomes the world
# X, Y and Z axes respectively. Use this rotation to estimate
# a homography that will be used to compute the output view.
# Apply the homography to generate the 3 fronto-parallel
# views and save them.
im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
Rt = get_rotation_matrix_rectification(R,'z')
H = get_homography(K,Rt)
fronto_im = get_fronto_image(H,im)
# fronto_im=Image.fromarray(fronto_im)
# fronto_im = fronto_im.rotate(180)
# fig = plt.figure()
# plt.imshow(fronto_im)
# cv2.imwrite('fronto_images_front.jpg', np.array(fronto_im))
cv2.imwrite('fronto_images_front.jpg', fronto_im,
                [int(cv2.IMWRITE_JPEG_QUALITY), 90])

Rt = get_rotation_matrix_rectification(R,'x')
H = get_homography(K,Rt)
fronto_im = get_fronto_image(H,im)
# fig = plt.figure()
# plt.imshow(fronto_im)
# cv2.imwrite('fronto_images_side.jpg', fronto_im)
cv2.imwrite('fronto_images_side.jpg', fronto_im,
                [int(cv2.IMWRITE_JPEG_QUALITY), 90])

Rt = get_rotation_matrix_rectification(R,'y')
H = get_homography(K,Rt)
fronto_im = get_fronto_image(H,im)
# fronto_im=Image.fromarray(fronto_im)
# # fronto_im = fronto_im.rotate(180)
# fig = plt.figure()
# plt.imshow(fronto_im)
# cv2.imwrite('fronto_images_top.jpg', np.array(fronto_im))
cv2.imwrite('fronto_images_top.jpg', fronto_im,
                [int(cv2.IMWRITE_JPEG_QUALITY), 90])